In [2]:
from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory

In [4]:
import os
import dotenv

dotenv.load_dotenv()



class Config:
    def __init__(self):
        
        self.telegram_token = os.getenv("TELEGRAM_BOT_TOKEN")



class AiConfig:
    def __init__(self, model_name: str = None,
                  model_kwargs: dict = None):
        
        self.model_name = "GigaChat-2-Max"
        self.model_kwargs = {}
        self.base_api_key = os.getenv("BASIC_API_KEY")
        self.access_token = None



config = Config()
aiconfig = AiConfig()


In [6]:
from langchain_community.chat_models.gigachat import GigaChat
import requests

from langchain_community.chat_message_histories import ChatMessageHistory, RedisChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate
import os
class AIGraph:
    def __init__(self, cfg: AiConfig):
        self.cfg = cfg



        self.model = self.initizialize_model()
        self.data = {}



    def _get_session_id(self, session_id: str) -> str:
        server_url = os.getenv('REDIS_URL')
        session_id = str(session_id)
        history = RedisChatMessageHistory(session_id=session_id, url=f'redis://{server_url}:6379')
        return history
    

    def _get_template(self):
        template = ChatPromptTemplate.from_messages(
            [
                (
                    'system',
                    "You are Jarvis Botz, an advanced AI assistant.\n"
                    "Act strictly as a {style}."
                ),
                ('placeholder', '{history}'),
                ('human', '{input}')
            ]
        )
        return template
    

    def _prepare_model_with_history(self, model):
        template = self._get_template()
        chain = template | model

        run = RunnableWithMessageHistory(chain, 
                                         get_session_history=self._get_session_id,
                                         input_messages_key='input', 
                                         output_messages_key='output',
                                         history_messages_key='history')
        
        
        return run



    def initizialize_model(self):
        access_token = self.get_access_token()
        model = GigaChat(model="GigaChat-2-Max", temperature=0.7,
                  access_token=access_token,
                  verify_ssl_certs=False
                  )
        model = self._prepare_model_with_history(model)
        return model



    def get_access_token(self):
        url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"

        payload={
        'scope': 'GIGACHAT_API_PERS'
        }

        headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': 'application/json',
        'RqUID': 'eb6b21c5-eb17-430c-9e56-ce024f69ca93',
        'Authorization': f'Basic {self.cfg.base_api_key}'
        }

        response = requests.request("POST", url, headers=headers, data=payload, verify=False)

        access_token = response.json().get("access_token")

        return access_token




graph = AIGraph(aiconfig)

c:\Users\odino\Desktop\jarvis_botz\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\odino\AppData\Local\Temp\ipykernel_24288\3112853290.py:58: LangChainDeprecationWarning: The class `GigaChat` was deprecated in LangChain 0.3.5 and will be removed in 1.0. An updated version of the class exists in the `langchain-gigachat package and should be used instead. To use it run `pip install -U `langchain-gigachat` and import as `from `langchain_gigachat import GigaChat``.
  model = GigaChat(model="GigaChat-2-Max", temperature=0.7,
